In [1]:
import pandas as pd
import os
import json
from datetime import datetime
from tqdm import tqdm
import math

In [2]:
def get_all_files_path_under(folder_path):
    
    all_jsons = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_jsons.append( os.path.normpath (file_path) )
    return all_jsons


In [3]:
from dateutil import parser
def time_difference(datetime_str1,datetime_str2):
    # Define your two datetime strings
    datetime_str1 = '2017-04-12T23:00:00Z'
    datetime_str2 = '2017-04-13T01:41:46Z'

    # Parse the datetime strings into datetime objects
    datetime_obj1 = parser.parse(datetime_str1)
    datetime_obj2 = parser.parse(datetime_str2)

    # Calculate the time difference
    time_difference = datetime_obj2 - datetime_obj1

    # Extract hours and minutes from the time difference
    hours, remainder = divmod(time_difference.total_seconds(), 3600)
    minutes = remainder / 60

    # Print the result
    return f"{int(hours)} hours and {int(minutes)} minutes"


In [4]:
def get_periodes(data):
    periods = data.get('liveData').get('linescore').get('periods')
    my_dict = {i: None for i in range(1,len(periods)+1)}
    for i in range(len(periods)):
        my_dict[i+1] = {'home': periods[i].get('home').get('rinkSide'), 'away':periods[i].get('away').get('rinkSide') }
    return my_dict

In [5]:
def home_or_away(shooterTeam,home,away):
    if shooterTeam == home:
        return str('home')
    if shooterTeam == away:
        return str('away')

In [6]:
def get_distance(eventPeriod,home_away,x,y,rink_info):
        if eventPeriod == 5:
            return 'shoot out'
        if home_away == 'home':
            rinkSide = rink_info[eventPeriod]['home']
            if rinkSide == 'right':
                return  math.sqrt((x +100)**2 + (y)**2)
            if rinkSide == 'left':
                return  math.sqrt((x -100)**2 + (y)**2)
        if home_away == 'away':
            rinkSide = rink_info[eventPeriod]['away']
            if rinkSide == 'right':
                return  math.sqrt((x +100)**2 + (y)**2)
            if rinkSide == 'left':
                return  math.sqrt((x -100)**2 + (y)**2)


In [7]:
data_folder = 'data/'
folder_path_20162017P = data_folder + '2016-2017/P'
folder_path_20162017R = data_folder +'2016-2017/R'

In [8]:
folder_path_20182019P = data_folder  + '2018-2019/P'
folder_path_20182019R = data_folder  + '2018-2019/R'

In [9]:
folder_path_20192020P = data_folder  + '2019-2020/P'
folder_path_20192020R = data_folder  + '2019-2020/R'

In [10]:
folder_path_20202021P = data_folder  + '2020-2021/P'
folder_path_20202021R = data_folder  + '2020-2021/R'

In [11]:
def create_csv(folder_path):
    data_to_tidy = get_all_files_path_under(folder_path)
    #columns = ['gamePk','season','gameType','gameStart_time','gameEnd_time','gameDuring','home','away',
 #'eventIdx','eventId','eventTypeId','shootType','shooter','goalie','eventPeriod','eventPeriodType','x','y'

#]
    #df = dd.from_pandas(pd.DataFrame(columns = columns), npartitions=1)
    data_list = []
    for j in tqdm(range(len(data_to_tidy))):
        json_file = data_to_tidy[j]

        with open(json_file) as json_file:
            data = json.load(json_file)

        gamePk = data.get('gamePk')

        gameData = data.get('gameData')
        game = gameData.get('game')
        season = game.get('season')
        gameType = game.get('type')
        gameStart_time = gameData.get('datetime').get('dateTime')
        gameEnd_time = gameData.get('datetime').get('endDateTime')
        gameDuring = time_difference(gameStart_time, gameEnd_time)

        teams = gameData.get('teams')
        home =  teams.get('home').get('name')
        away =  teams.get('away').get('name')
        
        rink_info = get_periodes(data)
        plays = data.get('liveData').get('plays').get('allPlays')
     
        for i in range(len(plays)):

            play = plays[i]
            about= play.get('about')
            res = play.get('result')
            eventIdx = about.get('eventIdx')
            eventId = about.get('eventId')
            eventTypeId = res.get('eventTypeId')
            eventPeriod = about.get('period')
            eventPeriodType = about.get('periodType')
            
       

            shootType = shooter = goalie =shooterTeam = shootResult= distance = None
            players = play.get('players')
              
            players = play.get('players')
            if eventTypeId == 'SHOT':
                shootType =  res.get('secondaryType')
                shooterTeam = play.get('team').get('name') 
                for player in players:
                    if player.get('playerType') == "Shooter":
                        shooter = player.get('player').get('fullName')
                    if  player.get('playerType') == "Goalie":
                        goalie = player.get('player').get('fullName')

            if eventTypeId == 'GOAL':
                shootType =  res.get('secondaryType') 
                shooterTeam = play.get('team').get('name') 
                for player in players:
                    if player.get('playerType') == 'Scorer':
                        shooter = player.get('player').get('fullName')
                    if  player.get('playerType') == "Goalie":
                        goalie = player.get('player').get('fullName') +'(failed)'

            if play.get('coordinates') == {}:
                            x = None
                            y = None
            else:
                            for coord in play.get('coordinates'):
                                if coord == 'x':
                                     x = play.get('coordinates')['x']
                                if coord == 'y':
                                     y = play.get('coordinates')['y']
            if shooterTeam is not None:  
                 if play.get('coordinates') != {}:
                    home_away = home_or_away(shooterTeam,home,away)   
                    distance =  get_distance(eventPeriod,home_away,x,y,rink_info)                            

            new_row = {'gamePk':gamePk,'season':season,'gameType':gameType,'gameStart_time':gameStart_time,
                       'gameEnd_time':gameEnd_time,'gameDuring':gameDuring,'home':home,'away':away,
                        'eventIdx':eventIdx,'eventId':eventId,'eventTypeId':eventTypeId,'shootType':shootType,
                       'shooter':shooter,'shooterTeam':shooterTeam,'distance':distance ,'goalie':goalie,
                       'eventPeriod':eventPeriod,
                       'eventPeriodType':eventPeriodType,'x':x,'y':y

                      }
            
            #df_to_concat = dd.from_pandas(pd.DataFrame(new_row,index=index)， npartitions=2)
            #df = dd.concat([df, df_to_concat])
            data_list.append(new_row)
    df = pd.DataFrame(data_list)        
    path =os.path.normpath(folder_path) 
    path.split('/')[-2:]
    csv_name = path.split('/')[-2]+'_'+ path.split('/')[-1]+'_clean'+'.csv'
    df.to_csv(csv_name, encoding='utf-8', index=False)
           


In [12]:
create_csv(folder_path_20162017P )

100%|██████████████████████████████████████████| 87/87 [00:00<00:00, 216.34it/s]


In [13]:
create_csv(folder_path_20162017R)

100%|██████████████████████████████████████| 1230/1230 [00:05<00:00, 244.04it/s]


In [14]:
create_csv(folder_path_20182019P)

100%|██████████████████████████████████████████| 87/87 [00:00<00:00, 230.17it/s]


In [15]:
create_csv(folder_path_20192020P)

100%|██████████████████████████████████████████| 86/86 [00:00<00:00, 207.98it/s]


In [16]:
create_csv(folder_path_20182019R)

100%|██████████████████████████████████████| 1271/1271 [00:05<00:00, 244.58it/s]


In [17]:
create_csv(folder_path_20192020R)

100%|██████████████████████████████████████| 1082/1082 [00:05<00:00, 207.13it/s]


In [18]:
create_csv(folder_path_20202021P)
create_csv(folder_path_20202021R)

100%|████████████████████████████████████████| 867/867 [00:03<00:00, 261.99it/s]


In [19]:
#df20162017R = pd.read_csv('2016-2017_R_clean.csv')
#df20162017P = pd.read_csv('2016-2017_P_clean.csv')

In [20]:
#df20182019P = pd.read_csv('2018-2019_P_clean.csv')

#df20192020R = pd.read_csv('2019-2020_R_clean.csv')
#df20192020P = pd.read_csv('2019-2020_P_clean.csv')

#df20202021R = pd.read_csv('2020-2021_R_clean.csv')
#df20202021P = pd.read_csv('2020-2021_P_clean.csv')

In [21]:
#df20182019 = pd.concat([df20182019R ,df20182019P])
#df20182019.to_csv('20182019_clean.csv', encoding='utf-8', index=False)

#df20192020 = pd.concat([df20192020R ,df20192020P])
#df20192020.to_csv('20192020_clean.csv', encoding='utf-8', index=False)

#df20202021 = pd.concat([df20202021R ,df20202021P])
#df20202021.to_csv('20202021_clean.csv', encoding='utf-8', index=False)

In [22]:
#df20182020 = pd.concat([df20182019 ,df20192020])
#df20182021 = pd.concat([df20182020 ,df20202021])

In [23]:
#df20182021.to_csv('20182021_clean.csv', encoding='utf-8', index=False)

In [24]:
#df20162017 = pd.concat([df20162017R,df20162017P])

In [25]:
#df20162017.to_csv('20162017_clean.csv', encoding='utf-8', index=False)